In [ ]:
import ollama
import time
import sys

MODEL_NAME = "llama3.2"

class OllamaClient:
    """Wraps Ollama API calls"""
    def __init__(self, model_name):
        self.model_name = model_name

    def send(self, prompt):
        """Send a user prompt to Ollama and return the model's reply
        Also prints timing info"""
        print("Sending to Ollama...")
        start = time.time()
        response = ollama.chat(model=self.model_name, messages=[{"role": "user", "content": prompt}])
        reply = response["message"]["content"]
        elapsed = time.time() - start
        print(f"Ollama responded in {elapsed:.2f} seconds")
        return reply

class ChatCLI:
    """A simple text-based chat interface using an OllamaClient"""

    def __init__(self, client):
        self.client = client

    def run(self):
        """Read user input and print bot replies
        Exits on 'quit' or CtrlC"""
        print("Text Chat Interface \ntype 'quit' or CtrlC to exit\n")
        try:
            while True:
                user_input = input("You: ").strip()
                if not user_input:
                    continue
                if user_input.lower() == "quit":
                    print("Goodbye!")
                    break

                bot_reply = self.client.send(user_input)
                print("Bot:", bot_reply, "\n")

        except (KeyboardInterrupt, EOFError):
            print("\nExiting")
            sys.exit(0)

client = OllamaClient(MODEL_NAME)
app = ChatCLI(client)
app.run()

In [ ]:
import ollama
import time
import sys
import os
import json

MODEL_NAME = "llama3.2"
MEMORY_FILE = "chat_memory.json"

class OllamaClient:
    """Wraps Ollama API calls"""
    def __init__(self, model_name, memory_file):
        self.model_name = model_name
        self.memory_file = memory_file
        #Load exisiting memory if present
        if os.path.exists(self.memory_file):
            with open(self.memory_file, "r") as f:
                self.history = json.load(f)
        else:
            self.history = [
                {"role": "system", "content": "You are helpful assistant"}
            ]
        
    def save_memory(self):
        with open(self.memory_file, "w") as f:
            json.dump(self.history, f, indent=2)

    def send(self, prompt):
        """Send a user prompt to Ollama and return the model's reply
        Also prints timing info"""
        self.history.append({"role": "user", "content": prompt})
        print("Sending to Ollama...")
        start = time.time()
        response = ollama.chat(model=self.model_name, messages=self.history)
        elapsed = time.time() - start
        print(f"Ollama responded in {elapsed:.2f} seconds")

        assitant_msg = response["message"]["content"]
        self.history.append({"role": "user", "content": assitant_msg})
        self.save_memory()

        return assitant_msg

class ChatCLI:
    """A simple text-based chat interface using an OllamaClient"""

    def __init__(self, client):
        self.client = client

    def run(self):
        """Read user input and print bot replies
        Exits on 'quit' or CtrlC"""
        print("Text Chat Interface \ntype 'quit' or CtrlC to exit\n")
        try:
            while True:
                user_input = input("You: ").strip()
                if not user_input:
                    continue
                if user_input.lower() == "quit":
                    print("Goodbye!")
                    break

                bot_reply = self.client.send(user_input)
                print("Bot:", bot_reply, "\n")

        except (KeyboardInterrupt, EOFError):
            print("\nExiting")
            sys.exit(0)

client = OllamaClient(MODEL_NAME, MEMORY_FILE)
app = ChatCLI(client)
app.run()